In [57]:
# Получение данных
import pandas as pd

file_name = './Hysys/data/hysys729_v2.csv'  # Откуда брать файл
data_source = pd.read_csv(file_name, sep=',', index_col=0,
                          skiprows=[1]  # Пропуск единиц измерения
                          ).dropna(
    axis=1, how='all')  # Чтение файла

column_names = data_source.columns.values # Получение названий столбцов
# pd.options.display.float_format = '{:,.2f}'.format # Если нужно ограничить вывод до 2 знаков после запятой

# Разбиение таблицы на параметры и критерии
params = data_source.iloc[:, 0:6].astype(float)
criteria1_column = data_source.iloc[:, 6].astype(float)
criteria2_column = data_source.iloc[:, 7].astype(float)
criteria3_column = data_source.iloc[:, 8].astype(float)

print('Размер таблицы:',data_source.shape)

ERROR_VALUE = -32767
if data_source[data_source == ERROR_VALUE].count().sum() > 0:
    print('В таблице есть битые данные')
    print(data_source[data_source == ERROR_VALUE].count())



Размер таблицы: (729, 9)


In [62]:
# Линейная интерполяция
from scipy.interpolate import LinearNDInterpolator
import numpy as np

# Создание точек для интерполяции
points = np.array(params.values)

PRINT_PROGRESS = False  # Вывод прогресса

X_LOWER = params.min().to_numpy()
X_UPPER = params.max().to_numpy()

# Создание интерполяторов
print('Расчет интерполяторов...')
print(f'Расчет интерполятора {column_names[6]}...')
criteria1 = LinearNDInterpolator(points, criteria1_column.values)
print(f'Расчет интерполятора {column_names[7]}...')
criteria2 = LinearNDInterpolator(points, criteria2_column.values)
print(f'Расчет интерполятора {column_names[8]}...')
criteria3 = LinearNDInterpolator(points, criteria3_column.values)
print('Расчет интерполяторов завершен')

bounds = pd.DataFrame([X_LOWER, X_UPPER],
                      columns=column_names[0:6],
                      index=['Нижняя граница', 'Верхняя граница'])
bounds.transpose()


Расчет интерполяторов...
Расчет интерполятора 10 - Master Comp Molar Flow (diM-Ether)...
Расчет интерполятора 14 - Master Comp Molar Flow (Methanol)...
Расчет интерполятора SPRDSHT-1 - B1:...
Расчет интерполяторов завершен


,Нижняя граница,Верхняя граница
4 - Temperature,140.0,170.0
5 - Temperature,240.0,260.0
6 - Temperature,340.0,360.0
8 - Temperature,90.0,110.0
VLV-100 - Pressure Drop,20.0,60.0
P-100 - Delta P,1000.0,2000.0


In [64]:
# Однокритериальная оптимизация
# Алгоритм Хука-Дживса
import pandas as pd
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch
from pymoo.core.problem import Problem
from pymoo.optimize import minimize


class Find_best_c1 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = -criteria1(x)


class Find_best_c2 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria2(x)


class Find_best_c3 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria3(x)


res_c1 = minimize(Find_best_c1(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c2 = minimize(Find_best_c2(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c3 = minimize(Find_best_c3(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
                  
# Сохранение результатов для однокритериальной оптимизации на основе маргинальных критериев
best_c1 = -res_c1.F
best_c2 = res_c2.F
best_c3 = res_c3.F

# Вывод в виде таблицы
single_criteria_optimization_df = pd.DataFrame(
    columns=column_names[0:6],
    data=[res_c1.X, res_c2.X, res_c3.X],
    index=column_names[6:9]
)
single_criteria_optimization_df['Значение критерия'] = [
    res_c1.F[0], res_c2.F[0], res_c3.F[0]]

print('Результаты однокритериальной оптимизации:')
single_criteria_optimization_df.transpose()
	

Результаты однокритериальной оптимизации:


,10 - Master Comp Molar Flow (diM-Ether),14 - Master Comp Molar Flow (Methanol),SPRDSHT-1 - B1:
4 - Temperature,140.00000,140.000000,1.400000e+02
5 - Temperature,260.00000,260.000000,2.600000e+02
6 - Temperature,360.00000,360.000000,3.500000e+02
8 - Temperature,110.00000,110.000000,1.100000e+02
VLV-100 - Pressure Drop,20.00000,20.000000,2.000000e+01
P-100 - Delta P,2000.00000,1000.000000,1.500000e+03
Значение критерия,-128.56923,1.989665,3.014509e+07


In [65]:
# Оптимизация на основе маргинальных решений

from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
import numpy as np


def marg_objective_function(x):
    '''Маргинальная целевая функция'''
    сurrent_c1 = criteria1(x)
    сurrent_c2 = criteria2(x)
    сurrent_c3 = criteria3(x)

    diff_c1 = best_c1 - сurrent_c1
    diff_c2 = сurrent_c2 - best_c2
    diff_c3 = сurrent_c3 - best_c3

    return np.sqrt(diff_c1**2 + diff_c2**2 + diff_c3**2)


# Постановка задачи однокритериальной оптимизации
class Marg_optimization(Problem):
    def __init__(self) -> None:
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = marg_objective_function(x)

# Запуск оптимизации
try:
    res_PS = minimize(
        Marg_optimization(), algorithm=PatternSearch(), seed=1)

    # Вывод результатов в виде таблицы
    margin_optimization_df = pd.DataFrame(
        data = (res_PS.X.tolist() + criteria1(res_PS.X).tolist() + criteria2(res_PS.X).tolist() + criteria3(res_PS.X).tolist()),
        index = column_names[0:9],
    )
except Exception as e:
    print('Ошибка маргинальной оптимизации')
    print(e)

print('Результаты маргинальной оптимизации')
margin_optimization_df

Результаты маргинальной оптимизации


,0
4 - Temperature,1.400000e+02
5 - Temperature,2.600000e+02
6 - Temperature,3.500000e+02
8 - Temperature,1.100000e+02
VLV-100 - Pressure Drop,2.000000e+01
P-100 - Delta P,1.500000e+03
10 - Master Comp Molar Flow (diM-Ether),1.282883e+02
14 - Master Comp Molar Flow (Methanol),2.622294e+00
SPRDSHT-1 - B1:,3.014509e+07


In [66]:
# Многокритериальная оптимизация
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Алгоритмы оптимизации
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.age import AGEMOEA

# Постановка задачи


class Multi_objective_optimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0,
                         # Минимальные значения параметров
                         xl=X_LOWER,
                         # Максимальные значения параметров
                         xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = -1 * criteria1(x)
        f2 = criteria2(x)
        f3 = criteria3(x)
        out["F"] = np.column_stack([f1, f2, f3])


# Запуск оптимизации
problem = Multi_objective_optimization()

try:
    print('Многокритериальная оптимизация (Метод NSGA2)')
    res_NSGA2 = minimize(problem, algorithm=NSGA2(), seed=1,
                         verbose=PRINT_PROGRESS, termination=('n_gen', 50))  # Вывод прогресса)
    # print('X: ', res_NSGA2.X)
    # print('F: ', res_NSGA2.F)

    # Сохранение результатов
    res_NSGA2_df = pd.DataFrame(res_NSGA2.X)
    res_NSGA2_df['dimEther_MCMF'] = -1 * res_NSGA2.F[:, 0]
    res_NSGA2_df['methanol_MCMF'] = res_NSGA2.F[:, 1]
    res_NSGA2_df['q_summ'] = res_NSGA2.F[:, 2]
    res_NSGA2_df.columns = column_names
    res_NSGA2_df.to_csv('NSGA2_Results.csv', sep=',', index=False,
                        header=data_source.columns.values)

except Exception as e:
    print('Ошибка NSGA2')
    print(e)

try:
    print('Многокритериальная оптимизация (Метод AGEMOEA)')
    res_AGEMOEA = minimize(problem, algorithm=AGEMOEA(),
                           seed=1, verbose=PRINT_PROGRESS,
                           # time based termination
                           termination=('time','00:03:00') # Не позволяем наглеть
                           )

    # Сохранение результатов
    res_AGEMOEA_df = pd.DataFrame(res_AGEMOEA.X)
    res_AGEMOEA_df['dimEther_MCMF'] = -1 * res_AGEMOEA.F[:, 0]
    res_AGEMOEA_df['methanol_MCMF'] = res_AGEMOEA.F[:, 1]
    res_AGEMOEA_df['q_summ'] = res_AGEMOEA.F[:, 2]
    res_AGEMOEA_df.columns = column_names
    res_AGEMOEA_df.to_csv('AGEMOEA_Results.csv', sep=',', index=False,
                          header=data_source.columns.values)
except Exception as e:
    print('Ошибка AGEMOEA')
    print(e)


Многокритериальная оптимизация (Метод NSGA2)
Многокритериальная оптимизация (Метод AGEMOEA)


In [ ]:
res_NSGA2_df
res_AGEMOEA_df